In [2]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked
np.random.seed(2498)


columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

In [3]:
len(dataset.item_id.unique()),dataset.item_id.unique().min(),dataset.item_id.unique().max()

(24507, 0, 24506)

In [4]:
from Data_manager.Build_Matrix.Data_import import build_ICM_type,build_ICM_lengh_kmeans_3,build_URM_ICM_onlyURM_item
import scipy.sparse as sps
URM_ALL,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
ICM_TYPE   =build_ICM_type(dataset_t)
ICM_LENGHT =build_ICM_lengh_kmeans_3(dataset_l,n_clusters = 5)
ICM_genre_subgenre = sps.hstack((ICM_TYPE, ICM_LENGHT));

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_ALL, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 697 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 308 ( 0.7%) Users that have less than 1 test interactions


In [6]:
param_1 = {'l1_ratio': 5.371794099882435e-05, 'topK': 1651, 'alpha': 0.008001600311161093, 'workers': 2, 'mw': 1}

In [7]:
p=MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked(URM_train,ICM_TYPE,ICM_LENGHT)

In [8]:
import os

output_folder_path = "result_MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 500
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
        "l1_ratio": Real(low = 1e-5, high = 0.1, prior = 'log-uniform'),
        "topK": Integer(200, 6000),
        "alpha": Real(low = 1e-5, high = 0.1, prior = 'uniform'),
        "workers": Categorical([5]),
        "mw": Integer(0, 10),
        "mw_1": Integer(0, 10)
    }

earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": URM_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM_TYPE,ICM_LENGHT],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [9]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

  0%|          | 0/24507 [00:00<?, ?it/s]

SearchBayesianSkopt: Resuming 'SLIMElasticNetRecommender'... Loaded 134 configurations.
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0010
Function value obtained: -0.0194
Current minimum: -0.0223
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 5.9859363001456074e-05, 'topK': 1310, 'alpha': 0.06120034908912765, 'workers': 5, 'mw': 0, 'mw_1': 9}


 15%|█▍        | 3560/24507 [02:30<13:35, 25.67it/s]

In [ ]:
from Recommenders.DataIO import DataIO
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import pandas as pd
recommender_class = SLIMElasticNetRecommender

output_folder_path = "result_experiments"
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
time_df = search_metadata["time_df"]

print('mean-time:')
print(time_df['train'].mean())


result_on_test_df = search_metadata["result_on_test_df"]
result_best_on_test = search_metadata["result_on_last"]
print('BEST_HYPERPARAMETERS: ')
print(search_metadata["hyperparameters_best"])
print(pd.DataFrame(result_on_test_df))
print(result_best_on_test)

mean-time:
392.422853257921
BEST_HYPERPARAMETERS: 
{'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001}
          PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
   cutoff                                                                      
0  10      0.048502                 0.067374  0.059069  0.022471    0.030846   
1  10           NaN                      NaN       NaN       NaN         NaN   
2  10           NaN                      NaN       NaN       NaN         NaN   
3  10           NaN                      NaN       NaN       NaN         NaN   
4  10           NaN                      NaN       NaN       NaN         NaN   
...             ...                      ...       ...       ...         ...   
95 10           NaN                      NaN       NaN       NaN         NaN   
96 10           NaN                      NaN       NaN       NaN         NaN   
97 10           NaN                      NaN       NaN       NaN         NaN   
98 10 